In [4]:
%%bash

# Download the dataset if it's not already there

if [ ! -f "./Appliances.json" ]; then
    curl "https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFiles/Appliances.json.gz" | gunzip > Appliances.json
else
    echo "Dataset already downloaded."
fi


Dataset already downloaded.


In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

In [3]:
df_raw = pd.read_json("Appliances.json", lines=True)

FileNotFoundError: File Appliances.json does not exist

In [ ]:
df_raw.memory_usage(deep=True).sum() / 1024**2

430.1570224761963

In [ ]:

df = df_raw.drop(['verified', 'reviewTime', 'reviewerID', 'asin', 'style', 'reviewerName', 'unixReviewTime', 'vote', 'image'], axis=1)

In [ ]:
df.dtypes

overall        int64
reviewText    object
summary       object
dtype: object

In [ ]:
df['overall'].describe()

count    602777.000000
mean          4.274209
std           1.301597
min           1.000000
25%           4.000000
50%           5.000000
75%           5.000000
max           5.000000
Name: overall, dtype: float64

In [ ]:
df["overall"] = df["overall"].astype("uint8")
df.memory_usage(deep=True).sum() / 1024**2

188.07587432861328

In [ ]:
print(f"Number of reviews: {len(df)}")

Number of reviews: 602777


In [ ]:
# create r_df from df by taking a random sample of 10,000 reviews
r_df = df.dropna().sample(n=1000, random_state=42)
r_df

s = r_df['reviewText'].values[:1000]

NameError: name 'df' is not defined

In [ ]:
import re
from collections import Counter

def get_vocab(texts):
    text = ' '.join(texts)
    c = Counter(re.findall('[a-z]+', text.lower()))
    return c.most_common(len(c))

d = get_vocab(s)


In [ ]:
import nltk

stop_words = set(nltk.corpus.stopwords.words('english'))

# remove stop words from my counter
d = [(k, v) for k, v in d if k not in stop_words]

In [ ]:
d

[('great', 233),
 ('one', 177),
 ('water', 170),
 ('works', 163),
 ('filter', 157),
 ('good', 147),
 ('fit', 123),
 ('time', 117),
 ('like', 117),
 ('price', 113),
 ('well', 113),
 ('dryer', 112),
 ('part', 108),
 ('easy', 97),
 ('would', 97),
 ('product', 97),
 ('use', 80),
 ('install', 80),
 ('perfect', 80),
 ('worked', 79),
 ('perfectly', 76),
 ('new', 76),
 ('replacement', 73),
 ('ice', 73),
 ('much', 73),
 ('get', 72),
 ('work', 71),
 ('original', 68),
 ('filters', 66),
 ('washer', 65),
 ('old', 64),
 ('bought', 59),
 ('buy', 57),
 ('replace', 55),
 ('dishwasher', 53),
 ('needed', 53),
 ('used', 52),
 ('right', 50),
 ('machine', 50),
 ('quality', 49),
 ('fridge', 49),
 ('love', 49),
 ('still', 48),
 ('months', 48),
 ('back', 48),
 ('little', 46),
 ('clean', 44),
 ('less', 44),
 ('door', 44),
 ('two', 43),
 ('ge', 43),
 ('years', 42),
 ('refrigerator', 42),
 ('happy', 42),
 ('first', 40),
 ('really', 40),
 ('exactly', 40),
 ('got', 39),
 ('way', 39),
 ('better', 39),
 ('came', 39),